# Make postISRCCD from raw for Spectractor StandAlone


- work with Weakly_2023_01
- use jupyter kernel LSST : **lsst_distrib_2023_01**



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/08
- last update : 2023/07/19



In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_27/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-6.0.0/eups 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_27
   gdf42428520+4e9f6d16c8 	current w_2023_27 setup


In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits




In [4]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [5]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [6]:
import lsst.daf.butler as dafButler

In [7]:
import warnings
warnings.filterwarnings("ignore")

# Configuration

In [8]:
FLAG_PLOT=False

In [9]:
DATE = 20230117
FILTER="empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"

#DATE = 20230118
#FILTER = "BG40_65mm_1-holo4_003"
#FILTER = "OG550_65mm_1-holo4_003"
#FILTER = "empty-holo4_003"

#DATE = 20230119
#FILTER="empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"


#DATE = 20230131
#FILTER = "empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"


#DATE = 20230201
#FILTER = "empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"


#DATE = 20230202
#FILTER = "empty-holo4_003"
#FILTER="BG40_65mm_1-holo4_003"
#FILTER="OG550_65mm_1-holo4_003"



#DATE = 20230704
#FILTER = "empty-holo4_003"

#DATE = 20230705
#FILTER = "empty-holo4_003"

#DATE = 20230706
#FILTER = "empty-holo4_003"


#filename_visits = f"visitdispersers_{DATE}_filt_{FILTER}.list"
filename_visits = f"all_visitdispersers/{DATE}/visitdispersers_{DATE}_filt_{FILTER}.list"

In [10]:
# output path of type top/date/filter
#----------------------------------------

top_path_out="my_postisrccd_img"
path_out=f"{top_path_out}/{DATE}"

In [11]:
if not os.path.exists(path_out):
    os.makedirs(path_out)

# read list of exposures

- generated by ListOfExposures-hologram.ipynb

In [12]:
df = pd.read_csv(filename_visits, header=None,skiprows = 1, sep=' ',names=["date","seq"])
df

,date,seq
0,20230202,643
1,20230202,651
2,20230202,652
3,20230202,660
4,20230202,661
5,20230202,674
6,20230202,675
7,20230202,683
8,20230202,684
9,20230202,692


In [13]:
for index,row in df.iterrows():
    exposure_selected =row["date"]*100000+row["seq"]
    print(exposure_selected)

2023020200643
2023020200651
2023020200652
2023020200660
2023020200661
2023020200674
2023020200675
2023020200683
2023020200684
2023020200692
2023020200693
2023020200701
2023020200702
2023020200710
2023020200711
2023020200719
2023020200720
2023020200728
2023020200729
2023020200737
2023020200738
2023020200746
2023020200747
2023020200755
2023020200756
2023020200764
2023020200765
2023020200773
2023020200774
2023020200782
2023020200783
2023020200791
2023020200792
2023020200800
2023020200801
2023020200814
2023020200815
2023020200828
2023020200829
2023020200837
2023020200838
2023020200846
2023020200847
2023020200860
2023020200861
2023020200874
2023020200875
2023020200883
2023020200884
2023020200892
2023020200893


## Select flags options

In [14]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Transformations
astropy scale transformations

In [15]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform = PercentileInterval(98.)

## Butler

In [16]:
#repo = '/sdf/group/rubin/repo/main'
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [17]:
collection='LATISS/raw/all'

In [18]:
# configuration
isr_config =  IsrTaskConfig()

In [19]:
isr_config.doDark = False
isr_config.doFlat =  False
isr_config.doFringe = False
isr_config.doDefect = True
isr_config.doLinearize = False
isr_config.doCrosstalk =  False
isr_config.doSaturationInterpolation = False
isr_config.overscan.fitType: 'MEDIAN_PER_ROW'
isr_config.doBias: True


In [20]:
isr_task = IsrTask(config=isr_config)

In [ ]:
###########################################
# REPO=/repo/embargo
# butler query-collections $REPO LATISS/calib
#############################################

In [21]:
calibType = 'bias'
physical_filter = 'empty~empty'
cameraName = 'LATISS'
# Collection name containing the verification outputs.

calibCollections = ['LATISS/calib','LATISS/raw/all',
'LATISS/calib/DM-28636',
'LATISS/calib/DM-28636/curated/19700101T000000Z',
'LATISS/calib/DM-28636/curated/20180101T000000Z',
'LATISS/calib/DM-28636/unbounded',
'LATISS/calib/DM-33875',
'LATISS/calib/DM-36484/bias.20221005a',
'LATISS/calib/DM-36484/biasGen.20221005a',
'LATISS/calib/DM-36484/biasGen.20221005a/20221006T000101Z',
'LATISS/calib/DM-36484/dark.20221006a',
'LATISS/calib/DM-36484/darkGen.20221005a',
'LATISS/calib/DM-36484/darkGen.20221005a/20221006T222501Z',
'LATISS/calib/DM-36484/darkGen.20221006a',
'LATISS/calib/DM-36484/darkGen.20221006a/20221006T222921Z',
'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a/20221007T002703Z',
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a/20221007T003732Z',
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a/20221007T004708Z',
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a/20221006T233657Z',
'LATISS/calib/DM-36484/verifyBias.20221005a',
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T000747Z',
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T213237Z',
'LATISS/calib/DM-36484/verifyBias.20221005b',
'LATISS/calib/DM-36484/verifyBias.20221005b/20221019T205236Z',
'LATISS/calib/DM-36484/verifyDark.20221006a',
'LATISS/calib/DM-36484/verifyDark.20221006a/20221006T224403Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a/20221007T003418Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a/20221007T004423Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a/20221006T234341Z',
'LATISS/calib/DM-36719',
'LATISS/calib/DM-36719/bias.20221107',
'LATISS/calib/DM-36719/biasGen.20221107a',
'LATISS/calib/DM-36719/biasGen.20221107a/20221107T205127Z',
'LATISS/calib/DM-36719/biasGen.20221107b',
'LATISS/calib/DM-36719/biasGen.20221107b/20221107T213306Z',
'LATISS/calib/DM-36719/dark.20221107',
'LATISS/calib/DM-36719/darkGen.20221107a',
'LATISS/calib/DM-36719/darkGen.20221107a/20221107T223409Z',
'LATISS/calib/DM-36719/flat-SDSSg.20221107',
'LATISS/calib/DM-36719/flat-SDSSi.20221107',
'LATISS/calib/DM-36719/flat-SDSSr.20221107',
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a/20221108T002737Z',
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a/20221108T005202Z',
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a/20221107T235401Z',
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a',
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a/20221108T180421Z',
'LATISS/calib/DM-36719/verifyBias.20221107b',
'LATISS/calib/DM-36719/verifyBias.20221107b/20221107T220410Z',
'LATISS/calib/DM-36719/verifyDark.20221107a',
'LATISS/calib/DM-36719/verifyDark.20221107a/20221107T232823Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a/20221108T004225Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T012110Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T014950Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a/20221108T000940Z',
'LATISS/calib/DM-39505',                    
'LATISS/calib/DM-39505/crosstalk.20230602',
'LATISS/calib/DM-38946',
'LATISS/calib/DM-38946/noRGseq/bias.20230503',
'LATISS/calib/DM-38946/noRGseq/dark.20230503',
'LATISS/calib/DM-38946/noRGseq/flat-g.20230503',
'LATISS/calib/DM-38946/noRGseq/flat-r.20230503',
'LATISS/calib/DM-38946/noRGseq/flat-i.20230503',
'u/czw/DM-37811/parOStest.20230202a/calib/flat-OG550.20230207a',
'u/czw/DM-37811/parOStest.20230202a/calib/flat-BG40.20230207a', 
'u/czw/DM-37811/parOStest.20230202a/calib/flat-SDSSr.20230203a',
'u/czw/DM-37811/parOStest.20230202a/calib/flat-SDSSg.20230203a',
'u/czw/DM-37811/parOStest.20230202a/calib/flat-SDSSi.20230202a',
'u/czw/DM-37811/parOStest.20230202a/calib/dark.20230202a',
'u/czw/DM-37811/parOStest.20230202a/calib/bias.20230202a',
'LATISS/calib/DM-37587/flat-BG40.20230113a',
'LATISS/calib/DM-37587/flat-OG550.20230113a',
'LATISS/calib/DM-37587/flat-SDSSr.20230113a',
'LATISS/calib/DM-36719',
'LATISS/calib/DM-36719/bias.20221107',
'LATISS/calib/DM-36719/dark.20221107',
'LATISS/calib/DM-36719/flat-SDSSi.20221107',
'LATISS/calib/DM-36719/flat-SDSSr.20221107',
'LATISS/calib/DM-36719/flat-SDSSg.20221107',
'LATISS/calib/DM-36484/bias.20221005a',
'LATISS/calib/DM-36484/dark.20221006a',
'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
'u/czw/defects.20220608',
'LATISS/calib/DM-33875',                                        
'u/czw/DM-28920/calib/bias.20210720',                 
'u/czw/DM-28920/calib/dark.20210720a',     
'u/calib/DM-32209-20211013a-g',
'u/calib/DM-32209-20211013a-felh',
'u/czw/DM-28920/calib/flat.20210720',
'u/czw/DM-28920/calib/defect.20210720a',
'LATISS/calib/DM-39635',
'LATISS/calib/DM-39635/unbounded','LATISS/defaults','LATISS/raw/all']

In [22]:
butler = dafButler.Butler(repo, collections=calibCollections)
camera = butler.get('camera', instrument=cameraName)
bias = butler.get('bias',instrument=cameraName,detector=0)
defects = butler.get('defects',instrument=cameraName,detector=0)


In [23]:
%matplotlib inline
if FLAG_PLOT:
    for index,row in df.iterrows():
        exposure_selected =row["date"]*100000+row["seq"]



        raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
        bias = butler.get("bias",instrument=cameraName, exposure= exposure_selected , detector=0, collections=calibCollections)
        defects = butler.get('defects',instrument=cameraName, exposure= exposure_selected ,detector=0,collections=calibCollections)
    
        meta = raw_img.getMetadata()
        md = meta.toDict()
    
        the_object = md['OBJECT']
        
        if the_object == "MU-COL":
            md['OBJECT']="HD38666"
            the_object = md['OBJECT']
            
        if the_object == "ETA1-DOR":
            md['OBJECT']="HD42525"
            the_object = md['OBJECT']
            
            
        the_am= md['AMSTART']
        the_filter=md['FILTER']
    
        #fast ISR 
        isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
        rotated_array = isr_img.exposure.image.array[::-1,::-1] #rotate the array 180 degrees
        #np.flip(np.flip(a, 1), 0)

    
        fig = plt.figure(figsize=(12,10))
        afw_display = afwDisplay.Display(frame=fig)
        afw_display.scale('linear', 'zscale',None)
        the_title = f"{index} :: isr exposure : {exposure_selected}, target={the_object}, airmass={the_am:.2f}, filter={the_filter}"
        afw_display.mtv(isr_img.exposure.image,title=the_title)

In [24]:
# Save in files

In [25]:
for index,row in df.iterrows():
    
    exposure_selected =row["date"]*100000+row["seq"]
    
    print(f"exposure selected = {exposure_selected}")


    raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    bias = butler.get("bias",instrument=cameraName, exposure= exposure_selected , detector=0, collections=calibCollections)
    defects = butler.get('defects',instrument=cameraName, exposure= exposure_selected ,detector=0,collections=calibCollections)
    
    
    isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
    arr=isr_img.exposure.image.array
    # 180 degree rotation
    rotated_array = arr[::-1,::-1] #rotate the array 180 degrees
    
    
    meta = raw_img.getMetadata()
    md = meta.toDict()

    the_object = md['OBJECT']
    the_am= md['AMSTART']
    the_filter=md['FILTER']
    
    if the_object == "MU-COL":
        md['OBJECT']="HD38666"
        the_object = md['OBJECT']

    filename_out = f"exposure_{exposure_selected}_pseudo-postisrccd.fits"
    fullfilename_out=os.path.join(path_out,filename_out)
    
    print(f">>>>  output filename {filename_out} object {the_object}")
    
    hdr = fits.Header()
    
    for key,value in md.items():
        if key == 'OBJECT':
            print(key,value)
        hdr[str(key)]=value
        
    # need this    
    hdr["AMEND"] = hdr["AMSTART"]
    
    # be aware weather data may be missing
    if hdr["AIRTEMP"] == None:
        hdr["AIRTEMP"] = 10.0

    if hdr["PRESSURE"] == None:
        hdr["PRESSURE"] = 744.

    if hdr["HUMIDITY"] == None:
        hdr["HUMIDITY"] = 50.

    if hdr["WINDSPD"] == None:
        hdr["WINDSPD"] = 5.

    if hdr["WINDDIR"] == None:
        hdr["WINDDIR"] = 0.   

    if hdr["SEEING"] == None:
        hdr["SEEING"] = 1.15
    
        
   
    
    # Be carefull for Spectractor, 2 hdu units are necessary
    
    primary_hdu = fits.PrimaryHDU(header=hdr)
    image_hdu = fits.ImageHDU(rotated_array)
    
    hdu_list = fits.HDUList([primary_hdu, image_hdu])
    
    hdu_list.writeto(fullfilename_out,overwrite=True)
    
    
    
    if index>=0:
        break
    


exposure selected = 2023020200643


>>>>  output filename exposure_2023020200643_pseudo-postisrccd.fits object HD185975
